In [ ]:
%load_ext autotime    

In [ ]:
import numpy as np
import importlib, os, datetime, pickle
from sus.protocol_designer import System, Protocol, Potential, Compound_Protocol
from sus.protocol_designer.protocol import sequential_protocol
from IPython import display
from IPython.display import HTML, Image
import matplotlib.animation as animation
from matplotlib.animation import FuncAnimation, PillowWriter
from quick_sim import setup_sim

# import edward_tools.fq_runner as fq_runner
from edward_tools.coupled_fq_potential import coupled_flux_qubit_non_linear_approx_pot, coupled_flux_qubit_non_linear_approx_force
from edward_tools.visualization import animate_sim_flux_qubit, plotFidelityBarChart, separate_by_state_2
from edward_tools.initial_state_sampling import extra_constraint_00_and_11_only
from NAND_PARAMETERS import *
import importlib

import kyle_tools as kt
import matplotlib.pyplot as plt


from edward_tools import coupled_fq_protocol_library, cfq_runner
from edward_tools import coupled_fq_protocol_library

import edward_tools.cfq_batch_sweep as cfq_batch_sweep
import edward_tools.Analysis_tool.general_analysis_tools as general_analysis_tool
# from edward_tools.Analysis_tool.general_analysis_tools import show_phi_dc_with_time
import edward_tools.Analysis_tool.minimum_value_of_potential as minimum_value_of_potential
from edward_tools.couple_flux_qubit_metrics import fidelityEvaluation
from edward_tools import visualization

from edward_tools.pot_analysis_helper_functions import get_XYU, pot_function, find_all_critical_points_for_all_potential

coupled_fq_protocol_library = importlib.reload(coupled_fq_protocol_library)
create_system = coupled_fq_protocol_library.create_system
get_potential_shot_at_different_t = coupled_fq_protocol_library.get_potential_shot_at_different_t
get_potential_shot_at_different_t_1D = coupled_fq_protocol_library.get_potential_shot_at_different_t_1D
create_simple_protocol_parameter_dict = coupled_fq_protocol_library.create_simple_protocol_parameter_dict
coupled_fq_runner = importlib.reload(cfq_runner)
coupled_fq_protocol_library = importlib.reload(coupled_fq_protocol_library)
create_system = coupled_fq_protocol_library.create_system
get_potential_along_a_1D_cutline = coupled_fq_protocol_library.get_potential_along_a_1D_cutline
plotCutlines = coupled_fq_protocol_library.plotCutlines

In [ ]:
    
fig = plt.figure(figsize= [20, 10])
ax_0, ax_1, ax_2, ax_3 = fig.add_subplot(2, 4, 1), fig.add_subplot(2, 4, 2), fig.add_subplot(2, 4, 3), fig.add_subplot(2, 4, 4)
ax_4, ax_5, ax_6, ax_7 = fig.add_subplot(2, 4, 5), fig.add_subplot(2, 4, 6), fig.add_subplot(2, 4, 7), fig.add_subplot(2, 4, 8) 
plt.close()

In [ ]:
def create_parameter_list(cfqr, _protocol):
    params_list_used = cfqr.protocol.get_params(0)
    params_list_used[8] = _protocol['phi_1x'] # phi_1x
    params_list_used[9] = _protocol['phi_2x'] # phi_2x"
    params_list_used[10] = _protocol['phi_1xdc'] # phi_1xdc
    params_list_used[11] = _protocol['phi_2xdc'] # phi_2xdc
    params_list_used[12] = _protocol['mu_12'] # mu
    return params_list_used

# parameters

### circuit parameters

In [ ]:
circuit_params_from_Chris = {
    "L": 5e-12 * 1, "T":4.2, "L_factor": 1, 
    "beta": 2.3, "d_beta": 0,
    "phi_1x": 0.61,   "phi_2x": 0.10, "phi_1xdc": 0, "phi_2xdc": 1.79, "mu_12": 0.06
}


circuit_params_chosen = circuit_params_from_Chris
experiment_comment = "Experiment 3 (2024-10-22): Failed KE extraction protocol simulation for Kyle's animation"

phi_1x_on_12, phi_2x_on_12, phi_1xdc_on_12, phi_2xdc_on_12, phi_1x_on_21, phi_2x_on_21, phi_1xdc_on_21, phi_2xdc_on_21, mu_12_on, four_well, CE_1, CE_5, CE_8, lower_H_wells, flip_H, lower_V_wells, flip_V, KE_extraction_protocol_for_CE_1, KE_extraction_protocol_for_CE_8 = generate_protocols_from_circuit_params(circuit_params_chosen)

In [ ]:
has_velocity = True

PHI_0 = 2.067833848 * 1e-15
k_B = 1.38e-23
T = circuit_params_chosen["T"]
k_BT = k_B * T

C_factor = 1
L_factor = circuit_params_chosen['L_factor']
R_factor = 80
# I_m_factor = 50
# I_m_factor = 15
I_m_factor = 0
time_scale = 1.0

I_p_1, I_p_2 = 5e-6 , 5e-6  # Amp
I_m_1, I_m_2 = 7e-9 * I_m_factor, 7e-9 * I_m_factor                           # Amp
R_1, R_2 = 1 * R_factor, 1 * R_factor                                         # ohm
C_1, C_2 = 500e-15 * C_factor, 500e-15 * C_factor                             # F

L_1, L_2 = 140e-12 * L_factor, 140e-12 * L_factor                             # H 
L_1, L_2 = 5e-12 * L_factor, 5e-12 * L_factor                             # H 
freq = 1/np.sqrt(C_1 * L_1)
characteristic_time = np.sqrt(C_1 * C_factor * L_1 * L_factor)

In [ ]:
m_c, m_1, m_2 = C_1, C_1, C_2
x_c = PHI_0 / (2 * np.pi)
time_scale_factor = 1
t_c = np.sqrt(L_1 * C_1)
v_c = x_c / t_c

U0_1 = m_c * x_c**2 / t_c**2 / k_BT
U0_2 = m_2 * x_c**2 / t_c**2 / k_BT
kappa_1, kappa_2, kappa_3, kappa_4 = 1/U0_1, 1/U0_1, 1/U0_1, 1/U0_1

lambda_1 = 2 * np.sqrt(L_1 * C_1) / (C_1 * R_1)
theta_1  = 1
eta_1    = np.sqrt(np.sqrt(L_1 * C_1)/ (R_1 * C_1)) * np.sqrt(2 * kappa_1 / 1**2)

lambda_2 = 2 * np.sqrt(L_1 * C_1) / (C_2 * R_2)
theta_2  = 1 / (C_2/C_1)
eta_2    = np.sqrt(np.sqrt(L_1 * C_1)/ (R_1 * C_1)) * np.sqrt(2 * kappa_2 * (R_1 * C_1**2) / (R_2 * C_2**2))

lambda_3 = 2 * np.sqrt(L_1 * C_1) / (C_1 * R_1)
theta_3  = 4
eta_3    = np.sqrt(np.sqrt(L_1 * C_1)/ (R_1 * C_1)) * np.sqrt(8 * kappa_3)

lambda_4 = 2 * np.sqrt(L_1 * C_1) / (C_2 * R_2)
theta_4  = 4 / (C_2/C_1)
eta_4    = np.sqrt(np.sqrt(L_1 * C_1)/ (R_1 * C_1)) * np.sqrt(8 * kappa_4 * (R_1 * C_1**2) / (R_2 * C_2**2))

gamma = 10

beta_1 = 2 * np.pi * L_1 * I_p_1 / PHI_0; 
beta_2 = 2 * np.pi * L_2 * I_p_2 / PHI_0;

beta_1, beta_2 = [2.0]*2
d_beta_1, d_beta_2 = 2 * np.pi * L_1 * I_m_1 / PHI_0, 2 * np.pi * L_2 * I_m_2 / PHI_0


_damping_factor = 1
_lambda = np.array([lambda_1, lambda_2, lambda_3, lambda_4])
_theta  = np.array([theta_1, theta_2, theta_3, theta_4])
_eta  =   np.array([eta_1, eta_2, eta_3, eta_4])

v_1 = np.random.normal(0, np.sqrt(k_BT/m_1)) / v_c
v_2 = np.random.normal(0, np.sqrt(k_BT/m_2)) / v_c
v_3 = np.random.normal(0, np.sqrt(k_BT/(m_1/4))) / v_c
v_4 = np.random.normal(0, np.sqrt(k_BT/(m_2/4))) / v_c

In [ ]:
print(f"L_1 = {L_1 * 1e12:.3g}pH, T = {T}K, U0_1 = {U0_1}")
print(f"freq = {freq / 1e9:.3g}GHz")
print(characteristic_time)
print(U0_1)

In [ ]:

def plot_3_well_potential(protocol_list, left_y_range, right_y_range, bottom_left_x_range = [-4, 4], bottom_right_x_range = [-4, 4], bottom_left_y_range = [-50, 500], bottom_right_y_range = [-50, 500], contour_x_range = [-4, 4], contour_y_range = [-4, 4], beta = beta_1, d_beta = 0, phi_2_left_fixed = -2.1, phi_2_right_fixed = 2.0, set_same_min = False):
    phi_2_array = np.linspace(-4, 4, 1000)
    params_list_used = cfqr.protocol.get_params(0)
    fig = plt.figure(figsize= [15, 7.5])
    ax_0, ax_1, ax_2, ax_3 = fig.add_subplot(2, 4, 1), fig.add_subplot(2, 4, 2), fig.add_subplot(2, 4, 3), fig.add_subplot(2, 4, 4)
    ax_4, ax_5, ax_6, ax_7 = fig.add_subplot(2, 4, 5), fig.add_subplot(2, 4, 6), fig.add_subplot(2, 4, 7), fig.add_subplot(2, 4, 8) 

    critical_point_list = []
    color_code = []

    X, Y = np.meshgrid(phi_2_array, phi_2_array)
    
    for _protocol in protocol_list:
        name     = _protocol['name']
        _protocol['beta'], _protocol['d_beta'] = beta, d_beta

        critical_points = find_all_critical_points_for_all_potential(_protocol, guess = [(-2, -2), (-2, 2), (3, -3), (2, 2)])
        critical_point_list.append(critical_points)
        params_list_used[4], params_list_used[5] = beta, beta # phi_1x
        params_list_used[8] = _protocol['phi_1x'] # phi_1x
        params_list_used[9] = _protocol['phi_2x'] # phi_2x"
        params_list_used[10] = _protocol['phi_1xdc'] # phi_1xdc
        params_list_used[11] = _protocol['phi_2xdc'] # phi_2xdc
        params_list_used[12] = _protocol['mu_12'] # mu

        params_at_t_str = [f"{key}: {value:.3g}" for key, value in zip(cfqr.protocol_key[-6:-1], params_list_used[-6:-1])]
        fig.suptitle(f"phi_1xdc = {params_list_used[10]:.3g}, phi_2xdc = {params_list_used[11]:.3g}, phi_1x = {params_list_used[8]:.3g}, phi_2x = {params_list_used[9]:.3g}, mu = {params_list_used[12]:.3g}") 

        potential_min = []
        U = cfqr.potential.potential(X, Y, _protocol['phi_1xdc'], _protocol['phi_2xdc'], params_list_used)
        potential = cfqr.potential.potential(phi_2_left_fixed, phi_2_array,  _protocol['phi_1xdc'], _protocol['phi_2xdc'], params_list_used)
        # print(potential)
        color_code.append(_protocol['color'])
        ax_0.plot(phi_2_array, potential, label = f"{name}", c = _protocol['color'], linestyle = _protocol['linestyle'])
        ax_4.plot(phi_2_array, potential - min(potential), label = f"{name}", c = _protocol['color'], linestyle = _protocol['linestyle'])

        potential = cfqr.potential.potential(phi_2_right_fixed, phi_2_array, _protocol['phi_1xdc'], _protocol['phi_2xdc'], params_list_used)
        ax_2.plot(phi_2_array, potential, label = f"{name}", c = _protocol['color'], linestyle = _protocol['linestyle'])
        ax_6.plot(phi_2_array, potential - min(potential), label = f"{name}", c = _protocol['color'], linestyle = _protocol['linestyle'])

        ax_0.set_xlim(-4, 4)
        ax_0.set_ylim(left_y_range[0], left_y_range[1])
        ax_4.set_xlim(bottom_left_x_range[0], bottom_left_x_range[1])
        ax_4.set_ylim(bottom_left_y_range[0], bottom_left_y_range[1])


        ax_2.set_xlim(-4, 4)
        ax_2.set_ylim(right_y_range[0], right_y_range[1])
        ax_6.set_xlim(bottom_right_x_range[0], bottom_right_x_range[1])
        ax_6.set_ylim(bottom_right_y_range[0], bottom_right_y_range[1])
        ax_0.legend()
        

    ax_1.contourf(X, Y, U, 30)
    ax_3.contourf(X, Y, U, 30)
    ax_1.vlines(x=phi_2_left_fixed, ymin = -4, ymax = 4, color = "red")
    ax_3.vlines(x=phi_2_right_fixed, ymin = -4, ymax = 4, color = "red")

    ax_5.contourf(X, Y, U, 30)
    ax_7.contourf(X, Y, U, 30)

    
    for critical_point_info, _c in zip(critical_point_list, color_code):
        for (_x, _y) in critical_point_info['coord']:
            ax_1.scatter(_x, _y, color = _c)
            ax_3.scatter(_x, _y, color = _c)
            ax_5.scatter(_x, _y, color = _c)
            ax_7.scatter(_x, _y, color = _c)
    
    ax_5.set_xlim(*contour_x_range)
    ax_7.set_xlim(*contour_x_range)
    ax_5.set_ylim(*contour_y_range)
    ax_7.set_ylim(*contour_y_range)

    plt.show() 
    return critical_point_list

### parameter setting

In [ ]:
"""
# step 0: modify parameters
- All the parameters are stored in a separate file PARAMETER_INPUT
- You can override some of the parameters here.
"""
params = {}
params['N'] = 1000
params['dt'] = 1/100
params['lambda'] = 1
params['beta'] = 1
params['sim_params'] = [_lambda, _theta, _eta]
params['target_work'] = None
params['applyOffset'] = False
params['measureWorkWithOffset'] = True
params['monitor_work_dist_in_whole_process'] = True # To monitor the work process
params['comment'] = experiment_comment
params['capacitance'] = np.array([C_1, C_2, C_1/4, C_2/4])
params['mass'] = np.array([1, 1, 1/4, 1/4])
params['v_c'] = x_c/t_c
params['k_BT'] = k_BT
params['U0'] = U0_1
params['as_step'] = np.s_[::10] # the time step to skep for the all_state
params['percentage'] = 1 # For what percentage of the total sample do you want to keep in the output all_state


In [ ]:
"""
# step 2: Define initial condition and protocol
"""
manual_domain=[np.array([-10, -10]), np.array([10, 10])]



phi_1x_init = 0.0
phi_2x_init = 0.032
mu_12_init = 0.016
phi_1xdc_init = 0.0
phi_2xdc_init =  1.8

initial_parameter_dict = {
        "U0_1": U0_1,     "U0_2": U0_2,     "gamma_1": gamma,  "gamma_2": gamma,
        "beta_1": beta_1,   "beta_2": beta_2,   "d_beta_1": d_beta_1 ,   "d_beta_2": d_beta_2,
        "phi_1x": phi_1x_init,  "phi_2x": phi_2x_init,  "phi_1xdc": phi_1xdc_init,  "phi_2xdc": phi_2xdc_init,
        "mu_12": mu_12_init, 'x_c': x_c
}



In [ ]:
"""
# step 1: Define potential
"""
coupled_fq_default_param = [1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, x_c]
[phi_1_bound, phi_2_bound, phi_1dc_bound, phi_2dc_bound] = np.array([4, 4, 4, 4])/time_scale_factor
contour_range = [300, 2000]
    
coupled_fq_domain = [[-phi_1_bound, -phi_2_bound, -phi_1dc_bound, -phi_2dc_bound], \
                     [phi_1_bound, phi_2_bound, phi_1dc_bound, phi_2dc_bound]]

coupled_fq_pot = Potential(coupled_flux_qubit_non_linear_approx_pot, coupled_flux_qubit_non_linear_approx_force, 14, 4,\
                           default_params = initial_parameter_dict,  relevant_domain = coupled_fq_domain)

### protocol setting

In [ ]:
zeroDissipation = False
# zeroDissipation = True

saveAllStates = True

params['sim_params'] = [_lambda, _theta, _eta]

if zeroDissipation:
    params['sim_params'] = [_lambda * 0, _theta, _eta * 0]

params['circuit_parameters'] = {
    "C_factor":C_factor, "L_factor": L_factor, "R_factor": R_factor, "I_m_factor": I_m_factor, "T": T, 
    "I_p_1": I_p_1, "I_p_2": I_p_2, "I_m_1": I_m_1, "I_m_2": I_m_2,
    "R_1": R_1, "R_2": R_2, "C_1": C_1, "C_2": C_2, "L_1": L_1, "L_2": L_2, 
    "characteristic_time": np.sqrt(C_1 * C_factor * L_1 * L_factor),
    "phi_1x_on": phi_1x_on_12, "phi_2x_on": phi_2x_on_12,
    "phi_1xdc_on": phi_1xdc_on_12, "phi_2xdc_on": phi_2xdc_on_12, "mu_12_on": mu_12_on,
    "gamma": gamma
}


initial_state_used_choice = ["four_well", "after_CE_8_completed"]

# bookmark
# initial_parameter_dict["phi_1xdc"] = 0
# initial_parameter_dict["phi_2xdc"] = 0
# initial_parameter_dict["phi_1x"]   = 0
# initial_parameter_dict["phi_2x"]   = 0
# initial_parameter_dict["mu_12"]      = 0

### modify the protocol

In [ ]:
mapping_state_1_to_state_2_dict_SWAP = {'00': ['01'], '01': ['00'], '10': ['10'], '11': ['11']}
mapping_state_1_to_state_2_dict_CE = {'00': ['01'], '01': ['01'], '10': ['10'], '11': ['11']}
mapping_state_1_to_state_2_dict_storage = {'00': ['00'], '01': ['01'], '10': ['10'], '11': ['11']}
mapping_dict = mapping_state_1_to_state_2_dict_storage

## acceleration protocol

In [ ]:
def pot_function(circuit_params, _phi_1dc = 0, _phi_2dc = 0, U_ratio = U0_1):
    _phi_1x = circuit_params["phi_1x"]
    _phi_2x = circuit_params["phi_2x"]
    _phi_1xdc = circuit_params["phi_1xdc"]
    _phi_2xdc = circuit_params["phi_2xdc"]
    _mu_12 = circuit_params["mu_12"]
    _xi = 1/(1-_mu_12**2)

    def Fcn(coord):
        _phi_1, _phi_2 = coord
        u1_1 = 1/2 * _xi * (_phi_1 - _phi_1x)**2
        u1_2 = 1/2 * gamma * (phi_1dc - _phi_1xdc)**2
        u1_3 = beta_1 * np.cos(_phi_1) * np.cos(_phi_1dc/2)
        u1_4 = -d_beta_1 * np.sin(_phi_1) * np.sin(_phi_1dc/2)

        u2_1 = 1/2 * _xi * (_phi_2 - _phi_2x)**2
        u2_2 = 1/2 * gamma * (phi_2dc - _phi_2xdc)**2
        u2_3 = beta_2 * np.cos(_phi_2) * np.cos(_phi_2dc/2)
        u2_4 = -d_beta_2 * np.sin(_phi_2) * np.sin(_phi_2dc/2)
        u3 = _mu_12 * _xi * (_phi_1 - _phi_1x) * (_phi_2 - _phi_2x)

        return U_ratio * (u1_1 + u1_2 +  u1_3 + u1_4 + u2_1 + u2_2 + u2_3 + u2_4 + u3)

    return Fcn


In [ ]:
# control swap protocol
def createProtocol(duration, protocol, name):
    protocol_new = {
        "phi_1xdc": protocol["phi_1xdc"], "phi_2xdc": protocol["phi_2xdc"], 
        "phi_1x": protocol["phi_1x"], "phi_2x": protocol["phi_2x"], 
        "mu_12": protocol["mu_12"], "duration": duration, "name": name
    }
    return protocol_new

In [ ]:
# control swap protocol
mu_factor_0 = 0.8
mu_factor_1 = 0.32
mu_factor_4 = 0.55
mu_factor_5 = 0.0
phi_2xdc_common = 1.80
# protocol_3 = {"phi_1x": 0.0, "phi_2x": -0.08, "mu_12": -0.04, "phi_1xdc": 0, "phi_2xdc": 1.96, "name": "deceleration_protocol_1", "color": "red", "linestyle" "-"}
# protocol_4 = {"phi_1x": 0.8, "phi_2x": -0.02, "mu_12": -0.01, "phi_1xdc": 0, "phi_2xdc": 1.96, "name": "deceleration_protocol_2", "color": "pink", "linestyle": "--"}
protocol_0 = {"phi_1x": 0.0, "phi_2x": 0.0, "mu_12": 0.0, "phi_1xdc": 0, "phi_2xdc": 0, "name": "four_well", "color": "cyan", "linestyle": "-"} 
protocol_1 = {"phi_1x": -0.0, "phi_2x": 0.0 * mu_factor_0, "mu_12": 0.0 * mu_factor_0, "phi_1xdc": 0, "phi_2xdc": phi_2xdc_common, "name": "pre_acceleration_protocol", "color": "orange", "linestyle": "-"}
protocol_2 = {"phi_1x": 0 * mu_factor_1, "phi_2x": 0.1 * mu_factor_1, "mu_12": 0.05 * mu_factor_1, "phi_1xdc": 0, "phi_2xdc": phi_2xdc_common, "name": "acceleration_protocol_0", "color": "g", "linestyle": "-"}

protocol_2_adiabatic = {"phi_1x": 0 * mu_factor_1, "phi_2x": 0.1 * mu_factor_1, "mu_12": 0.05 * mu_factor_1, "phi_1xdc": 0, "phi_2xdc": 1.85, "name": "acceleration_protocol_0", "color": "g", "linestyle": "-"}
# protocol_4 = {"phi_1x": 0, "phi_2x": 0.1 * mu_factor_4, "mu_12": 0.05 * mu_factor_4, "phi_1xdc": 0, "phi_2xdc": 0, "name": "deceleration_protocol_0", "color": "cyan", "linestyle": "--"}
protocol_5 = {"phi_1x": 0, "phi_2x": -0.1 * mu_factor_5, "mu_12": -0.05 * mu_factor_5, "phi_1xdc": 0, "phi_2xdc": phi_2xdc_common, "name": "deceleration_protocol_1", "color": "b", "linestyle": "--"}


In [ ]:
# control swap protocol
mu_factor_0 = 0.4
mu_factor_1 = 1.1
mu_factor_4 = 0.55
mu_factor_5 = 0.
phi_2xdc_common = 1.82
# protocol_3 = {"phi_1x": 0.0, "phi_2x": -0.08, "mu_12": -0.04, "phi_1xdc": 0, "phi_2xdc": 1.96, "name": "deceleration_protocol_1", "color": "red", "linestyle" "-"}
# protocol_4 = {"phi_1x": 0.8, "phi_2x": -0.02, "mu_12": -0.01, "phi_1xdc": 0, "phi_2xdc": 1.96, "name": "deceleration_protocol_2", "color": "pink", "linestyle": "--"}
protocol_0 = {"phi_1x": 0.0, "phi_2x": 0.0, "mu_12": 0.0, "phi_1xdc": 0, "phi_2xdc": 0, "name": "four_well", "color": "cyan", "linestyle": "-"} 
protocol_1a = {"phi_1x": -0.0, "phi_2x": 0.0 * mu_factor_0, "mu_12": 0.00 * mu_factor_0, "phi_1xdc": 0, "phi_2xdc": phi_2xdc_common, "name": "pre_acceleration_protocol", "color": "orange", "linestyle": "-"}
protocol_1 = {"phi_1x": -0.0, "phi_2x": 0.1 * mu_factor_0, "mu_12": 0.05 * mu_factor_0, "phi_1xdc": 0, "phi_2xdc": phi_2xdc_common, "name": "pre_acceleration_protocol", "color": "orange", "linestyle": "-"}
protocol_2 = {"phi_1x": 0 * mu_factor_1, "phi_2x": 0.1 * mu_factor_1, "mu_12": 0.05 * mu_factor_1, "phi_1xdc": 0, "phi_2xdc": 1.86, "name": "acceleration_protocol_0", "color": "g", "linestyle": "-"}
# protocol_4 = {"phi_1x": 0, "phi_2x": 0.1 * mu_factor_4, "mu_12": 0.05 * mu_factor_4, "phi_1xdc": 0, "phi_2xdc": 0, "name": "deceleration_protocol_0", "color": "cyan", "linestyle": "--"}
protocol_5 = {"phi_1x": 0, "phi_2x": -0.1 * mu_factor_5, "mu_12": -0.05 * mu_factor_5, "phi_1xdc": 0, "phi_2xdc": 1.68, "name": "deceleration_protocol_1", "color": "red", "linestyle": "--"}
# protocol_6 = {"phi_1x": 0, "phi_2x": 0, "mu_12": 0 * mu_factor_5, "phi_1xdc": 0, "phi_2xdc": 1.90, "name": "deceleration_protocol_1", "color": "red", "linestyle": "--"}
protocol_7 = {"phi_1x": phi_1x_init, "phi_2x": phi_2x_init, "mu_12": mu_12_init, "phi_1xdc": phi_1xdc_init, "phi_2xdc": phi_2xdc_init, "name": "acceleration_protocol_1", "color": "red", "linestyle": "--"}

In [ ]:
protocol_array = [protocol_0, protocol_1, protocol_2, protocol_5]
print(U0_1)

critical_point_list = plot_3_well_potential(protocol_array, [500, 1200], [800,1200], bottom_left_y_range= [-10, 200], bottom_right_y_range= [-10, 50], contour_x_range = [-2.5, -1.5], contour_y_range = [1.2, 1.8], 
                                             beta = beta_1, phi_2_left_fixed=-2, phi_2_right_fixed= 2, set_same_min=True)

## escape rate 

In [ ]:
t_c

In [ ]:
index_longitudinal = 1
index_transverse = 0
for _list, _protocol in zip(critical_point_list, protocol_array):
    params_list_used = cfqr.protocol.get_params(0)
    params_list_used[4], params_list_used[5] = beta_1, beta_2 # phi_1x
    params_list_used[8] = _protocol['phi_1x'] # phi_1x
    params_list_used[9] = _protocol['phi_2x'] # phi_2x"
    params_list_used[10] = _protocol['phi_1xdc'] # phi_1xdc
    params_list_used[11] = _protocol['phi_2xdc'] # phi_2xdc
    params_list_used[12] = _protocol['mu_12'] # mu
    print("-" * 20)
    print(_protocol['name'])
    omega_list, delta_U_list, at_weak_list = [], [], []
    _phi_1xdc, _phi_2xdc = _protocol['phi_1xdc'], _protocol['phi_2xdc']
    saddle_point_x, saddle_point_y = _list['coord'][2][0], 0
    saddle_point_potential = coupled_flux_qubit_non_linear_approx_pot(saddle_point_x, saddle_point_y, _phi_1xdc, _phi_2xdc, params_list_used)

    for _pt in [_list['coord'][2], _list['coord'][3], (saddle_point_x, 0)]:
        print(f"coord and phi = : {_pt}, {_phi_1xdc}, {_phi_2xdc}")
        k_eff = np.array([1 - beta_1 * np.cos(_pt[0]) * np.cos(_phi_1xdc/2), 1 - beta_2 * np.cos(_pt[1]) * np.cos(_phi_2xdc/2)]) 
        omega = np.sqrt(k_eff/ t_c**2)
        omega_list.append(omega)
        print(f"omega = {omega}, k_eff = {k_eff}")

        well_potential = coupled_flux_qubit_non_linear_approx_pot(_pt[0], _pt[1], _phi_1xdc, _phi_2xdc, params_list_used)
        delta_U = saddle_point_potential - well_potential
        delta_U_list.append(delta_U)
        print(f"delta_U = {delta_U}, well_potential = {well_potential}, saddle_point_potential = {saddle_point_potential}")

    omega_10 = omega_list[0][0] * omega_list[0][1] / omega_list[2][0]
    omega_11 = omega_list[1][0] * omega_list[1][1] / omega_list[2][0]
    G = 1 / (R_1 * C_1 * omega_list[2][0])
    a_t = (1 + G**2 / 4) - G/2
    a_t_weak = 2 * np.pi / (omega_10 * R_1 * C_1) * (delta_U_list[0])**2
    # print(omega_10 * R_1 * C_1)

    escape_rate_10 =  omega_10 / (2 * np.pi) * np.exp(-delta_U_list[0])
    escape_rate_11 = omega_11 / (2 * np.pi) * np.exp(-delta_U_list[1]) 
    print(f"dU_10 = {delta_U_list[0]}, dU_11 = {delta_U_list[1]}, a_t = {a_t}, a_t_weak = {a_t_weak}")
    print(f"escape_rate_10: {escape_rate_10}, escape_rate_11: {escape_rate_11}")
    print("-" * 50)


## time adjust

In [ ]:
pre_acceleration_potential =  createProtocol(60, protocol_1, name = "pre_acceleration_potential")
acceleration_potential_jump =  createProtocol(0.01, protocol_2, name = "acceleration_potential_jump")
acceleration_potential =  createProtocol(20, protocol_2, name = "acceleration_potential")
# decceleration_potential_jump = createProtocol(0.01, protocol_5, name = "decceleration_potential")
decceleration_potential_jump =  createProtocol(0.1, protocol_1a, name = "pre_acceleration_potential")
decceleration_potential = createProtocol(1.0, protocol_5, name = "decceleration_potential")
decceleration_potential =  createProtocol(20, protocol_1a, name = "pre_acceleration_potential")


acceleration_potential_for_adiabatic =  createProtocol(20, protocol_2, name = "acceleration_potential")

protocol_list_momentum = [ createProtocol(20, protocol_7, name = "acceleration_potential") ] 

protocol_list_adiabatic = [
    createProtocol(60, protocol_2_adiabatic, name = "acceleration_potential"),
    createProtocol(5, protocol_2_adiabatic, name = "acceleration_potential_hold"),
    {'duration': 80.0, "phi_1x": 0.0, "phi_2x": 0.0, "mu_12": 0.00, "phi_1xdc": 0, "phi_2xdc": 0, "name": "four_well"}
]  


def tellMeImportantFactors(protocol):
    print(f"N = {params['N']}, dt = {1/params['dt']}")
    print(f"T = {T} K, L = {L_1} H, beta = {beta_1}")
    print(f"R = {R_1} Ω, C = {C_1} F")
    print(f"phi_1x = {phi_1x_on_12}, phi_2x = {phi_2x_on_12}, phi_1xdc = {phi_1xdc_on_12}, phi_2xdc = {phi_2xdc_on_12}, mu_12 = {mu_12_on}")
    for x in protocol:
        print(x)

In [ ]:
protocol_list = protocol_list_momentum
# protocol_list = protocol_list_adiabatic
# protocol_list = protocol_list_check_suddenly_change_phi_xdc
# protocol_list = protocol_list_check_the_limit_for_error 


In [ ]:
for x in protocol_list_momentum:
    print(x)

# setting protocol (click here!)

# first initialization

### create initial state 

In [ ]:
initial_parameter_dict

In [ ]:
%%capture
initial_state_array = ["saved_init_state/four_well_default_init_state_N_1000.npy", "saved_init_state/after_CE_8_completed_N_1000z.npy", "saved_init_state/just_before_KEP_N_1000.npy"]
initial_state_array = [f"saved_init_state/four_well_default_init_state_N_1000_T={T}, L={L_1}, beta = {beta_1}.npy"]
initial_state_array = [f"saved_init_state/four_well_default_init_state_N_1000_T={T}, L={L_1}, beta = {beta_1}, phi_2xdc = {phi_2xdc}.npy"]
initial_state_chosen = initial_state_array[0]


regenerate_init_state = False
init_state_saved = None
# regenerate_init_state = True
if regenerate_init_state:
    initial_potential_protocol_list = [create_CE_Protocol(200*100*params['dt'], four_well)]
    init_params = params.copy()
    init_params['dt'] = 1/500
    init_state_saved = cfq_batch_sweep.create_initial_state(initial_parameter_dict, initial_potential_protocol_list, coupled_fq_pot, init_params)
    np.save(f"saved_init_state/four_well_default_init_state_N_1000_T={T}, L={L_1}, beta = {beta_1}, phi_2xdc = {phi_2xdc}.npy", init_state_saved)
else:
    init_state_saved = np.load(initial_state_chosen)

init_state_used = init_state_saved

# particle_index = visualization.separate_by_state_2(init_state_saved[:, (0, 1), :])
# init_state_used = init_state_saved[particle_index["00"]]

In [ ]:
# np.save(f"saved_init_state/four_well_default_init_state_N_1000_T={T}, L={L_1}, beta = {beta_1}, phi_2xdc = {phi_2xdc}.npy", final_state)
# init_state_used = final_state


In [ ]:
init_state_saved_new = init_state_saved[...]
init_state_saved_new[:, 0, 0] = -2
init_state_saved_new[:, 1, 0] = -2
init_state_saved_new[:, 2, 0] = 0
init_state_saved_new[:, 3, 0] = phi_2xdc_init
init_state_used = init_state_saved_new

In [ ]:
plt.scatter(init_state_used[..., 0, 0], init_state_used[..., 1, 0])
plt.xlim(-4, 4)
plt.ylim(-4, 4)

In [ ]:
import edward_tools
importlib.reload(edward_tools)
find_all_critical_points_for_all_potential = edward_tools.pot_analysis_helper_functions.find_all_critical_points_for_all_potential

### create cfqr object

In [ ]:
computation_protocol_parameter_dict

In [ ]:
"""
# step 3: create the relevant storage protocol and computation protocol
"""
def modifiedFunction(t, result):
    return result


computation_protocol_parameter_dict = coupled_fq_protocol_library.customizedProtocol(initial_parameter_dict, \
                                                                    protocol_list)
storage_protocol, comp_protocol = create_system(computation_protocol_parameter_dict, modifiedFunction = None)

cfqr = cfq_runner.coupledFluxQubitRunner(potential = coupled_fq_pot, params = params, \
                                                storage_protocol= storage_protocol, \
                                                computation_protocol= comp_protocol, \
                                         protocol_list = protocol_list, \
                                        has_velocity=has_velocity)

protocol_time_array, protocol_time_index_array, protocol_all_time_array, protocol_time_all_index_array = cfqr.createProtocolTimeArray(protocol_list, params)
cfqr.initialize_sim()
# cfqr.set_sim_attributes(init_state=init_state_saved)
cfqr.set_sim_attributes(init_state=init_state_used)
print(f"mean KE = {np.mean(cfqr.system.get_kinetic_energy(init_state_used))}")

# initial_state_PE_and_KE

In [ ]:
plt.hist(cfqr.system.get_potential(init_state_used, 0), bins=50)

# actual simulations

In [ ]:
manual_domain=[np.array([-5, -5]), np.array([5, 5])]

print(f"characteristic_time = {characteristic_time * 10e9:.3g}ns, total speed = {1/(characteristic_time * protocol_time_array[-1]) / 1e9:.3g}GHz")

simResult = cfq_batch_sweep.simulateSingleCoupledFluxQubit(params, initial_parameter_dict, protocol_list, \
                        potential = coupled_fq_pot, potential_default_param = coupled_fq_default_param, mapping_state_1_to_state_2_dict = mapping_dict,         
                        initial_state = init_state_used, manual_domain = manual_domain, \
                        phi_1xdc = phi_1xdc,  phi_2xdc = phi_2xdc, measure_all_states=saveAllStates, \
                        has_velocity = has_velocity)


In [ ]:
f"{simResult['cfqr'].protocol_time_array[-1]:.3g}"

In [ ]:
critical_time_index = []
for x in simResult['cfqr'].protocol_time_array:
    critical_time_index.append(np.sum(simResult['cfqr'].getReducedTimeArray()<x))
    

In [ ]:
particle_index = simResult['cfqr'].getIndexOfParticles()
pColor = {"00": "#061DF7", "01": "#FCEF51", "10": "#3FC7F2", "11": "#F187F4"}
final_state = simResult['cfqr'].get_all_state()[:, -1,...]

for key in pColor.keys():
    plt.scatter(final_state[particle_index[key], 0, 0], final_state[particle_index[key], 1, 0], c = pColor[key])
plt.xlim(-4, 4)
plt.ylim(-4, 4)

In [ ]:
plt.hist(cfqr.system.get_kinetic_energy(final_state), bins = 100)

In [ ]:
simResult['fidelity']

In [ ]:
# particle_index = simResult['cfqr'].getIndexOfParticles()
# pColor = {"00": "#061DF7", "01": "#FCEF51", "10": "#3FC7F2", "11": "#F187F4"}

# fig, ax = plt.subplots(1, 3, figsize = [15, 5])

# for i, x in enumerate(critical_time_index):
#     for key in pColor.keys():
#         state = simResult['cfqr'].get_all_state()[particle_index[key],x,...]
#         ax[i].scatter(state[:,0,0], state[:,1,0], c = pColor[key])
#         ax[i].set_aspect(1)
#         ax[i].set_xlim(-4, 4)
#         ax[i].set_ylim(-4, 4)

In [ ]:
print(f"characteristic_time = {characteristic_time * 10e9:.3g}ns, freq = {1/characteristic_time * 10e-9:.3g} GHz")

In [ ]:
for x in protocol_list:
    print(x)

In [ ]:
# particle_index = simResult['cfqr'].getIndexOfParticles()
# pColor = {"00": "#061DF7", "01": "#FCEF51", "10": "#3FC7F2", "11": "#F187F4"}

# for key in pColor.keys():
#     plt.scatter(all_states_t_1[particle_index[key], 0, 0], all_states_t_1[particle_index[key], 1, 0], c = pColor[key])
# plt.xlim(-4, 4)
# plt.ylim(-4, 4)

In [ ]:
def getParamsList(_protocol):
    params_list_used = cfqr.protocol.get_params(0)
    params_list_used[8] = _protocol['phi_1x'] # phi_1x
    params_list_used[9] = _protocol['phi_2x'] # phi_2x"
    params_list_used[10] = _protocol['phi_1xdc'] # phi_1xdc
    params_list_used[11] = _protocol['phi_2xdc'] # phi_2xdc
    params_list_used[12] = _protocol['mu_12'] # mu
    return params_list_used

In [ ]:
def plot_state_and_potential(protocol_list, state_array, left_y_range = [-50, 500], right_y_range = [-50, 500], bottom_left_y_range = [-50, 500], bottom_right_y_range = [-50, 500], set_same_min = False):
    pColor = {"00": "#061DF7", "01": "#FCEF51", "10": "#3FC7F2", "11": "#F187F4"}
    
    params_list_1 = getParamsList(protocol_list[0])
    params_list_2 = getParamsList(protocol_list[1])
    
    particle_index = simResult['cfqr'].getIndexOfParticles()
    particle_index_00_01 = np.logical_or(particle_index['00'], particle_index['01'])
    particle_index_10_11 = np.logical_or(particle_index['10'], particle_index['11'])
    
    _state_before, _state_after = state_array[0], state_array[1]
    position_before, position_after = np.transpose(_state_before[..., 0]), np.transpose(_state_after[..., 0])
    _phi_2_before, _phi_2_after = position_before[1, :], position_after[1, :]
    
    
    _potential_of_particle_before = coupled_flux_qubit_non_linear_approx_pot(*position_before, params = params_list_1)
    _potential_of_particle_after  = coupled_flux_qubit_non_linear_approx_pot(*position_after, params = params_list_2)

    _phi_1_before_mean_left = np.mean(position_before[0, particle_index_00_01])
    _phi_1_before_right = np.mean(position_before[0, particle_index_10_11])

    _phi_1_after_mean_left  = np.mean(position_after[0, particle_index_00_01])
    _phi_1_after_mean_right = np.mean(position_after[0, particle_index_10_11])

    phi_2_array = np.linspace(-4, 4, 1000)
    
    
    fig = plt.figure(figsize= [15, 7.5])
    ax_0, ax_1, ax_2, ax_3 = fig.add_subplot(2, 4, 1), fig.add_subplot(2, 4, 2), fig.add_subplot(2, 4, 3), fig.add_subplot(2, 4, 4)

    # potential along the cutline
    potential_1_left  = cfqr.potential.potential(_phi_1_before_mean_left, phi_2_array, params_list_1[10], params_list_1[11], params_list_1)
    potential_1_right = cfqr.potential.potential(_phi_1_before_right, phi_2_array, params_list_1[10], params_list_1[11], params_list_1)

    
    potential_2_left = cfqr.potential.potential(_phi_1_after_mean_left, phi_2_array, params_list_2[10], params_list_2[11], params_list_2)
    potential_2_right = cfqr.potential.potential(_phi_1_after_mean_right, phi_2_array, params_list_2[10], params_list_2[11], params_list_2)
    
    

    # params_at_t_str = [f"{key}: {value:.3g}" for key, value in zip(cfqr.protocol_key[-6:-1], params_list_used[-6:-1])]
    # fig.suptitle(f"phi_1xdc = {params_list_used[10]:.3g}, phi_2xdc = {params_list_used[11]:.3g}, phi_1x = {params_list_used[8]:.3g}, phi_2x = {params_list_used[9]:.3g}, mu = {params_list_used[12]:.3g}") 

    # color_code.append(_protocol['color'])
    ax_0.plot(phi_2_array, potential_1_left, label = f"{protocol_list[0]['name']}", c = protocol_list[0]['color'], linestyle = protocol_list[0]['linestyle'])
    ax_0.plot(phi_2_array, potential_2_left, label = f"{protocol_list[1]['name']}", c = protocol_list[1]['color'], linestyle = protocol_list[1]['linestyle'])
    ax_0.set_ylim(left_y_range[0], left_y_range[1])
    ax_0.set_title(f"phi_1 = left")
    # ax_4.plot(phi_2_array, potential_1 - min(potential_1), label = f"{name}", c = _protocol['color'], linestyle = _protocol['linestyle'])

    ax_2.plot(phi_2_array, potential_1_right, label = f"{protocol_list[0]['name']}", c = protocol_list[0]['color'], linestyle = protocol_list[0]['linestyle'])
    ax_2.plot(phi_2_array, potential_2_right, label = f"{protocol_list[1]['name']}", c = protocol_list[1]['color'], linestyle = protocol_list[1]['linestyle'])
    ax_2.set_ylim(right_y_range[0], right_y_range[1])
    ax_2.set_title(f"phi_1 = right")

    # ax_1.vlines(x=-2, ymin = -4, ymax = 4, color = "red")
    # ax_3.vlines(x=2, ymin = -4, ymax = 4, color = "red")
    

    work_done = _potential_of_particle_after - _potential_of_particle_before

    ax_0.scatter(_phi_2_before[particle_index['00']], _potential_of_particle_before[particle_index['00']], alpha = 0.002, c = pColor["00"])
    ax_0.scatter(_phi_2_before[particle_index['01']], _potential_of_particle_before[particle_index['01']], alpha = 0.002, c = pColor["01"])
    ax_0.scatter(_phi_2_after[particle_index['00']], _potential_of_particle_after[particle_index['00']], alpha = 1, c = pColor["00"])
    ax_0.scatter(_phi_2_after[particle_index['01']], _potential_of_particle_after[particle_index['01']], alpha = 1, c = pColor["01"])
    ax_1.hist(work_done[particle_index['00']], color = pColor["00"])
    ax_1.hist(work_done[particle_index['01']], color = pColor["01"])


    ax_2.scatter(_phi_2_before[particle_index['10']], _potential_of_particle_before[particle_index['10']], alpha = 0.002, c = pColor["10"])
    ax_2.scatter(_phi_2_before[particle_index['11']], _potential_of_particle_before[particle_index['11']], alpha = 0.002, c = pColor["11"])
    ax_2.scatter(_phi_2_after[particle_index['10']], _potential_of_particle_after[particle_index['10']], alpha = 1, c = pColor["10"])
    ax_2.scatter(_phi_2_after[particle_index['11']], _potential_of_particle_after[particle_index['11']], alpha = 1, c = pColor["11"])
    ax_3.hist(work_done[particle_index['10']], color = pColor["10"])
    ax_3.hist(work_done[particle_index['11']], color = pColor["11"])

    
    # for critical_point_info, _c in zip(critical_point_list, color_code):
    #     for (_x, _y) in critical_point_info['coord']:
    #         ax_1.scatter(_x, _y, color = _c)
    #         ax_3.scatter(_x, _y, color = _c)
    
    

    plt.show() 

In [ ]:
t_1_index = 400
t_1 = simResult['cfqr'].getReducedTimeArray()[t_1_index]
all_states_t_1 = simResult['cfqr'].get_all_state()[:, t_1_index, ...]

plot_state_and_potential([protocol_1, protocol_2], [all_states_t_1, all_states_t_1], left_y_range = [450, 1200], right_y_range = [700, 1500])


In [ ]:
pColor = {"00": "#061DF7", "01": "#FCEF51", "10": "#3FC7F2", "11": "#F187F4"}
mu_ratio = 0.1
protocol_5_test = {'phi_1x': 0, 'phi_2x': -0.1 * mu_ratio, 'mu': -0.05 * mu_ratio, 'phi_1xdc': 0, 'phi_2xdc': 1.90, 'name': 'deceleration_protocol_1', 'color': 'b', 'linestyle': '--'}
t_before_decelerate_index = 300
t_before_decelerate = simResult['cfqr'].getReducedTimeArray()[t_before_decelerate_index]
all_states_t_before_decelerate = simResult['cfqr'].get_all_state()[:, t_before_decelerate_index, ...]

plot_state_and_potential([protocol_2, protocol_5], [all_states_t_before_decelerate, all_states_t_before_decelerate], left_y_range = [700, 1000], right_y_range = [800, 1100])


In [ ]:
particle_index = simResult['cfqr'].getIndexOfParticles()

_phi_2_before_00 = all_states_t_before_decelerate[particle_index['00'], 1, 0]
_phi_2_before_01 = all_states_t_before_decelerate[particle_index['01'], 1, 0]
_KE_before_00 = simResult['cfqr'].system.get_kinetic_energy(all_states_t_before_decelerate)[particle_index['00']]
_KE_before_01 = simResult['cfqr'].system.get_kinetic_energy(all_states_t_before_decelerate)[particle_index['01']]
plt.scatter(_phi_2_before_00, _KE_before_00, c = pColor['00'])
plt.scatter(_phi_2_before_01, _KE_before_01, c = pColor['01'])
# plt.xlim(-4, 4)

# sim Results

### fidelity

In [ ]:
import sys, importlib
# importlib.reload(sys.modules['edward_tools.jupyter_helper_function'])
from edward_tools.jupyter_helper_function import work_analysis, KE_analysis, single_particle_analysis, fidelity_analysis, work_done_of_each_type
fidelity_analysis(simResult, params, initial_parameter_dict, protocol_list)
print(simResult['simulation_data']['pickle_save_path'])
print(t_c)

In [ ]:
simResult['fidelity']

# work done of each type

In [ ]:
work_done_of_each_type(simResult)

In [ ]:
tellMeImportantFactors(protocol_list)

###  single particle analysis

In [ ]:
import sys, importlib
importlib.reload(sys.modules['edward_tools.jupyter_helper_function'])
from edward_tools.jupyter_helper_function import work_analysis, KE_analysis, single_particle_analysis, fidelity_analysis, work_done_of_each_type
all_states = simResult['cfqr'].get_all_state()

particle_category = ["00"]
p_index = 6

for _c in particle_category:
    fig, ax1 = plt.subplots(figsize=(8, 5))
    ax2 = ax1.twinx()
    index_choice = simResult['cfqr'].getIndexOfParticles()[_c]
    reduced_data = single_particle_analysis(simResult, params, protocol_list, plot_ax = ax1, 
                             particle_index=p_index, particle_category= _c, 
                             time_range=None, selected_quantity = ['KE'])
    reduced_protocal_time_array = reduced_data["reduced_protocal_time_array"]
    index_slice = reduced_data["index_slice"]

    # print(simResult['cfqr'].protocol_time_array)
    y_min, y_max = ax1.set_ylim()
    x_min, x_max = ax1.set_xlim()
    for _t in simResult['cfqr'].protocol_time_array:
        plt.vlines(x=_t, ymin = y_min, ymax = y_max, linestyles="--")


    all_states_00_i1 = all_states[index_choice][p_index, ...]
    mean_phi_2 = all_states_00_i1[:, 1, 0]
    # mean_phi_1 = np.mean(all_states[index_choice][:, :, 0, 0], axis = 0)


    ax2.plot(reduced_protocal_time_array, mean_phi_2, alpha =0.8, color = "grey", label = f"avg phi_2 of {particle_category}")
    ax2.set_ylim(-4, 4)
    ax2.set_xlim(50, 120)
    ax2.set_ylabel(r"$\varphi_{2}$")

    ax2.hlines(1.5, x_min, x_max, color = "red", linestyle = "--")
    ax2.hlines(0.5, x_min, x_max, color = "red", linestyle = "--")
    ax1.set_title(f"energy profile of {particle_category} particle (total time of protocol = {protocol_time_array[-1] * characteristic_time * 1e9:.3g} ns)")
    ax2.legend()
    plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(8, 5))
all_states = simResult['cfqr'].get_all_state()
index_choice = simResult['cfqr'].getIndexOfParticles()["00"]
targetTimeArray = simResult['cfqr'].getReducedTimeArray()
ax.plot(targetTimeArray, np.mean(all_states[index_choice, :, 1, 0], axis = 0), label = "mean")
# ax.plot(np.std(all_states[index_choice, :, 0, 0], axis = 0))
ax.plot(targetTimeArray, np.max(all_states[index_choice, :, 1, 0], axis = 0), label = "max")
ax.plot(targetTimeArray, np.min(all_states[index_choice, :, 1, 0], axis = 0),label = "min")

max_phi_2_particle = np.argmax(all_states[index_choice, 600, 1, 0])
min_phi_2_particle = np.argmax(all_states[index_choice, 600, 1, 0])

max_KE = np.max(simResult['cfqr'].system.get_kinetic_energy(all_states[index_choice][max_phi_2_particle]))
ax.plot(targetTimeArray, simResult['cfqr'].system.get_kinetic_energy(all_states[index_choice][max_phi_2_particle])/max_KE - 1.0)
# plt.plot(targetTimeArray, simResult['cfqr'].system.get_kinetic_energy(all_states[index_choice][min_phi_2_particle]))
ax.set_title(f"max KE = {max_KE}")
ax.set_xlim(50, 70)
ax.set_ylim(-2, 0)


In [ ]:
np.max(simResult['cfqr'].system.get_kinetic_energy(all_states))

In [ ]:
protocol_array = [protocol_1, protocol_2, protocol_5]


critical_point_list = plot_3_well_potential(protocol_array, [0, 2000], [0, 2000], 
                                             bottom_left_x_range = [-1, 1], bottom_left_y_range= [100, 300], bottom_right_y_range= [-10, 320], contour_x_range = [-2.5, -1.5], contour_y_range = [1.2, 1.8], 
                                             beta = 2.0, phi_2_left_fixed=-2, phi_2_right_fixed= 2, set_same_min=True)

In [ ]:
max_phi_2_particle = np.argmax(all_states[index_choice, 600, 1, 0])
min_phi_2_particle = np.argmax(all_states[index_choice, 600, 1, 0])
plt.plot(targetTimeArray, simResult['cfqr'].system.get_kinetic_energy(all_states[index_choice][max_phi_2_particle]))
plt.plot(targetTimeArray, simResult['cfqr'].system.get_kinetic_energy(all_states[index_choice][min_phi_2_particle]))


In [ ]:
import sys, importlib
importlib.reload(sys.modules['edward_tools.jupyter_helper_function'])
from edward_tools.jupyter_helper_function import work_analysis, KE_analysis, single_particle_analysis, fidelity_analysis, work_done_of_each_type
all_states = simResult['cfqr'].get_all_state()
reducedTimeArray = simResult['cfqr'].getReducedTimeArray()

particle_category = ["00", "01"]

fig, ax1 = plt.subplots(figsize=(8, 5))
ax2 = ax1.twinx()

for _c in particle_category:

    index_choice = simResult['cfqr'].getIndexOfParticles()[_c]
    
    ax1.plot(reducedTimeArray, np.mean(all_states[index_choice, :, 1, 0], axis = 0))
    ax1.set_ylim(1.8, 2.1)
    ax1.set_xlim(5, 7.5)
    print(simResult['cfqr'].protocol_time_array)
    y_min, y_max = ax1.set_ylim()
    x_min, x_max = ax1.set_xlim()
    for _t in simResult['cfqr'].protocol_time_array:
        plt.vlines(x=_t, ymin = y_min, ymax = y_max, linestyles="--")


    all_states_00_i1 = all_states[index_choice][p_index, ...]
    mean_phi_2 = all_states_00_i1[:, 1, 0]
    # mean_phi_1 = np.mean(all_states[index_choice][:, :, 0, 0], axis = 0)


    ax2.plot(reduced_protocal_time_array, mean_phi_2, alpha =0.8, color = "grey", label = f"avg phi_2 of {particle_category}")
    ax2.set_ylim(-4, 4)
    ax2.set_ylabel(r"$\varphi_{2}$")

    ax2.hlines(1.5, x_min, x_max, color = "red", linestyle = "--")
    ax2.hlines(0.5, x_min, x_max, color = "red", linestyle = "--")
    ax1.set_title(f"energy profile of {particle_category} particle (total time of protocol = {protocol_time_array[-1] * characteristic_time * 1e9:.3g} ns)")
    ax2.legend()
    plt.show()

## KE histogram

In [ ]:
time_index_start = 0
time_index_end = -1
frame_skip =10
all_state = simResult['cfqr'].sim.output.all_state['states'][:, time_index_start:time_index_end, ...]
targetTimeArray = protocol_all_time_array[simResult['cfqr'].sim.target_step_index][time_index_start:time_index_end]

animation_time_array = targetTimeArray[::frame_skip]
animation_states = all_state[:, ::frame_skip, ...]

fig, ax = plt.subplots(2, 2, figsize=[10, 10])
ax_flatten = ax.flatten()


particle_index = simResult['cfqr'].getIndexOfParticles()

KE_01_int = simResult['cfqr'].system.get_kinetic_energy(animation_states[particle_index['00'], 0])
hist_plot = ax_flatten[0].hist(KE_01_int)[2]



def animate(i):
    params_at_t_str = [f"{key}: {value:.3g}" for key, value in zip(simResult['cfqr'].protocol_key[-6:-1], simResult['cfqr'].protocol.get_params(animation_time_array[i])[-6:-1])]
    fig.suptitle(f"t = {animation_time_array[i]} + {params_at_t_str}")
    ax_flatten[0].clear()
    
    ax_flatten[0].hist(simResult['cfqr'].system.get_kinetic_energy(animation_states[particle_index['00'], i]))
            
ani = FuncAnimation(fig, animate, interval=500, frames=len(animation_time_array))
writer = animation.PillowWriter(fps=5, metadata=dict(artist='Me'), bitrate=1800)
ani.save('scatter.gif', writer=writer)
plt.close()




In [ ]:
Image(filename="scatter.gif")

In [ ]:
all_state = simResult['cfqr'].sim.output.all_state['states']
targetTimeArray = protocol_all_time_array[simResult['cfqr'].sim.target_step_index]

fig, ax = plt.subplots(2, 2, figsize = [10, 10])
ax_flatten = [ax[1][0], ax[0][0], ax[1][1], ax[0][1]]
for i, key in enumerate(particle_index.keys()):
    max_KE = np.max(simResult['cfqr'].system.get_kinetic_energy(all_state[particle_index[key]]), axis = 0)
    min_KE = np.min(simResult['cfqr'].system.get_kinetic_energy(all_state[particle_index[key]]), axis = 0)
    mean_KE = np.mean(simResult['cfqr'].system.get_kinetic_energy(all_state[particle_index[key]]), axis = 0)
    ax_flatten[i].plot(targetTimeArray, min_KE)
    ax_flatten[i].plot(targetTimeArray, max_KE)
    ax_flatten[i].plot(targetTimeArray, mean_KE)
    ax_flatten[i].set_title(f"max and min KE of {key}")

## animation in different space

In [ ]:
all_states = simResult['cfqr'].get_all_state()
frame_skip = 20
targetTimeArray = protocol_all_time_array[simResult['cfqr'].sim.target_step_index]
animation_time_array = targetTimeArray[::frame_skip]
animation_states = all_states[:, ::frame_skip, ...]

fig, ax = plt.subplots(2, 5, figsize=[20, 7])
plot_label_1 = [("phi_1", "phi_2"), ("phi_1dc", "phi_2dc"), ("phi_1", "phi_1dc"), ("phi_1", "phi_2dc"), ("phi_2", "phi_1dc"), ("phi_2", "phi_2dc")]
plot_label_2 = ["phi_1", "phi_2", "phi_1dc", "phi_2dc"]
pColor = {"00": "#061DF7", "01": "#FCEF51", "10": "#3FC7F2", "11": "#F187F4"}

ax_flatten = ax.flatten()
ax_flatten_1 = [ax_flatten[i] for i in [0, 1, 2, 5, 6, 7]]
ax_flatten_2 = [ax_flatten[i] for i in [3, 4, 8, 9]]
phi_1_array = np.linspace(-4, 4, 1000)
phi_2_array = np.linspace(-4, 4, 1000)
phi_1dc_array = np.linspace(-4, 4, 1000)
phi_2dc_array = np.linspace(-4, 4, 1000)

for _ax, (xlabel, ylabel) in zip(ax_flatten_1, plot_label_1):
    _ax.set_xlim(-4, 4)
    _ax.set_ylim(-4, 4)
    _ax.set_xlabel(xlabel)
    _ax.set_ylabel(ylabel)    
    _ax.set_aspect(1)

for _ax, xlabel in zip(ax_flatten_2, plot_label_2):
    _ax.set_xlim(-4, 4)
    _ax.set_ylim(800, 1000)
    _ax.set_xlabel(xlabel)
    _ax.set_ylabel("PE")    
    # _ax.set_aspect(1)


plot_axis_1 = [[0, 1], [2, 3], [0, 2], [0, 3], [1, 2], [1, 3]]
plot_axis_2 = [0, 1, 2, 3]

scatter_array_1 = []
scatter_array_2 = []
potential_min_plot_array = {}
potential_max_plot_array = {}

potential_min_2_plot_array = {}
potential_max_2_plot_array = {}

potential_min_1dc_plot_array = {}
potential_max_1dc_plot_array = {}

# for key, index in particle_index.items():
for key, index in [("11", particle_index["11"])]:
    for _i, (_x, _y) in enumerate(plot_axis_1):
        _scatter = ax_flatten_1[_i].scatter(all_states[index, 0, _x, 0], all_states[index, 0, _y, 0], c = pColor[key])
        scatter_array_1.append(_scatter)

    for _i in plot_axis_2:
        potential = simResult['cfqr'].system.get_potential(animation_states[index, 0, ...], 0)
        _scatter = ax_flatten_2[_i].scatter(all_states[index, 0, _i, 0], potential, c = pColor[key])
        scatter_array_2.append(_scatter)
        
        params_at_0 = simResult['cfqr'].protocol.get_params(0)
        phi_1_min_array = np.min(animation_states[index, 0, 0, 0], axis = 0)
        phi_1_max_array = np.max(animation_states[index, 0, 0, 0], axis = 0)
        phi_1_mean = np.mean(animation_states[index, 0, 0, 0])

        phi_2_min_array = np.min(animation_states[index, 0, 1, 0], axis = 0)
        phi_2_max_array = np.max(animation_states[index, 0, 1, 0], axis = 0)
        phi_2_mean = np.mean(animation_states[index, 0, 1, 0])

        phi_1dc_min_array = np.min(animation_states[index, 0, 2, 0], axis = 0)
        phi_1dc_max_array = np.max(animation_states[index, 0, 2, 0], axis = 0)
        phi_1dc_mean = np.mean(animation_states[index, 0, 2, 0])

        
        phi_2dc_min_array = np.min(animation_states[index, 0, 3, 0], axis = 0)
        phi_2dc_max_array = np.max(animation_states[index, 0, 3, 0], axis = 0)
        phi_2dc_mean = np.mean(animation_states[index, 0, 3, 0], axis = 0)    
        
        if _i == 0:
            potential_min_array = coupled_flux_qubit_non_linear_approx_pot(phi_1_array, phi_2_mean, phi_1dc_mean, phi_2dc_mean, params_at_0)
            _potetnial_min_plot = ax_flatten_2[_i].plot(phi_1_array, potential_min_array, c = pColor[key], linestyle= "--")[0]
            potential_min_plot_array[key] = _potetnial_min_plot
        
            potential_max_array = coupled_flux_qubit_non_linear_approx_pot(phi_1_array, phi_2_mean, phi_1dc_mean, phi_2dc_mean, params_at_0)
            _potetnial_max_plot = ax_flatten_2[_i].plot(phi_1_array, potential_max_array, c = pColor[key])[0]
            potential_max_plot_array[key] = _potetnial_max_plot
            
        
        if _i == 1:
            print(phi_1_mean, phi_2_min_array, 0, 1.88)
            potential_min_array = coupled_flux_qubit_non_linear_approx_pot(phi_1_mean, phi_2_array, phi_1dc_mean, phi_2dc_mean, params_at_0)
            _potetnial_min_plot = ax_flatten_2[_i].plot(phi_2_array, potential_min_array, c = pColor[key], linestyle= "--")[0]
            potential_min_2_plot_array[key] = _potetnial_min_plot
        
            potential_max_array = coupled_flux_qubit_non_linear_approx_pot(phi_1_mean, phi_2_array, phi_1dc_mean, phi_2dc_mean, params_at_0)
            _potetnial_max_plot = ax_flatten_2[_i].plot(phi_2_array, potential_max_array, c = pColor[key])[0]
            potential_max_2_plot_array[key] = _potetnial_max_plot
        

        if _i == 2:
            potential_min_array = coupled_flux_qubit_non_linear_approx_pot(phi_1_min_array, phi_2_array, phi_1dc_array, phi_2dc_array, params_at_0)
            _potetnial_min_plot = ax_flatten_2[_i].plot(phi_1dc_array, potential_min_array, c = pColor[key], linestyle= "--")[0]
            potential_min_1dc_plot_array[key] = _potetnial_min_plot
        
            potential_max_array = coupled_flux_qubit_non_linear_approx_pot(phi_1_max_array, phi_2_array, phi_1dc_array, phi_2dc_array, params_at_0)
            _potetnial_max_plot = ax_flatten_2[_i].plot(phi_1dc_array, potential_max_array, c = pColor[key])[0]
            potential_max_1dc_plot_array[key] = _potetnial_max_plot
        
def animate(i):
    counter_1, counter_2 = 0, 0
    _t = animation_time_array[i]
    simResult['cfqr'].protocol.get_params(animation_time_array[i])
    params_at_t_str = [f"{key}: {value:.3g}" for key, value in zip(simResult['cfqr'].protocol_key[-6:-1], simResult['cfqr'].protocol.get_params(animation_time_array[i])[-6:-1])]
    fig.suptitle(f"t = {animation_time_array[i]} + {params_at_t_str}")
    
    # for key, index in particle_index.items():
    for key, index in [("11", particle_index["11"])]:
        for _i, (_x, _y) in enumerate(plot_axis_1):
            scatter_array_1[counter_1].set_offsets(np.c_[animation_states[index, i, _x, 0], animation_states[index, i, _y, 0]])
            counter_1 += 1

        for _i in plot_axis_2:
            potential_i = simResult['cfqr'].system.get_potential(animation_states[index, i, ...], animation_time_array[i])
            scatter_array_2[counter_2].set_offsets(np.c_[animation_states[index, i, _i, 0], potential_i])

            params_at_t = simResult['cfqr'].protocol.get_params(_t)
            phi_1_min_array = np.min(animation_states[index, i, 0, 0], axis = 0)
            phi_1_max_array = np.max(animation_states[index, i, 0, 0], axis = 0)
            
            phi_2_min_array = np.min(animation_states[index, i, 1, 0], axis = 0)
            phi_2_max_array = np.max(animation_states[index, i, 1, 0], axis = 0)
            

            phi_1dc_min_array = np.min(animation_states[index, i, 2, 0], axis = 0)
            phi_1dc_max_array = np.max(animation_states[index, i, 2, 0], axis = 0)
            
            phi_2dc_array = np.mean(animation_states[index, i, 3, 0], axis = 0)
            
            if _i == 0:    
                potential_min_array = coupled_flux_qubit_non_linear_approx_pot(phi_1_array, phi_2_array, phi_1dc_min_array, phi_2dc_array, params_at_t)
                potential_min_plot_array[key].set_xdata(phi_1_array)
                potential_min_plot_array[key].set_ydata(potential_min_array)
                
                potential_max_array = coupled_flux_qubit_non_linear_approx_pot(phi_1_array, phi_2_array, phi_1dc_max_array, phi_2dc_array, params_at_t)
                potential_max_plot_array[key].set_xdata(phi_1_array)
                potential_max_plot_array[key].set_ydata(potential_max_array)


            if _i == 1:    
                potential_min_array = coupled_flux_qubit_non_linear_approx_pot(phi_1_array, phi_2_min_array, phi_1dc_array, phi_2dc_array, params_at_t)
                potential_min_2_plot_array[key].set_xdata(phi_1_array)
                potential_min_2_plot_array[key].set_ydata(potential_min_array)
                
                potential_max_array = coupled_flux_qubit_non_linear_approx_pot(phi_1_array, phi_2_max_array, phi_1dc_array, phi_2dc_array, params_at_t)
                potential_max_2_plot_array[key].set_xdata(phi_1_array)
                potential_max_2_plot_array[key].set_ydata(potential_max_array)

            if _i == 2:
                potential_min_array = coupled_flux_qubit_non_linear_approx_pot(phi_1_min_array, phi_2_array, phi_1dc_array, phi_2dc_array, params_at_t)
                potential_min_1dc_plot_array[key].set_xdata(phi_1dc_array)
                potential_min_1dc_plot_array[key].set_ydata(potential_min_array)
                
                potential_max_array = coupled_flux_qubit_non_linear_approx_pot(phi_1_max_array, phi_2_array, phi_1dc_array, phi_2dc_array, params_at_t)
                potential_max_1dc_plot_array[key].set_xdata(phi_1dc_array)
                potential_max_1dc_plot_array[key].set_ydata(potential_max_array)
                
            
            counter_2 += 1

# ani = FuncAnimation(fig, animate, interval=100, frames=len(animation_time_array))
# writer = animation.PillowWriter(fps=30, metadata=dict(artist='Me'), bitrate=1800)
# ani.save('scatter.gif', writer=writer)
# # plt.close()

In [ ]:
Image(filename="scatter.gif")

In [ ]:
all_state = simResult['cfqr'].get_all_state()
reducedTimeArray = simResult['cfqr'].getReducedTimeArray()
particle_01_jump_to_a_wrong_well = all_state[particle_index["11"]][np.any(all_state[particle_index["11"]][:, :, 1, 0] < 0, axis = 1)]


plt.plot(particle_01_jump_to_a_wrong_well[0, :, 1, 0])
# plt.plot(simResult['cfqr'].system.get_kinetic_energy(particle_01_jump_to_a_wrong_well))

In [ ]:
PE_of_wrong_particle = [simResult["cfqr"].system.get_potential(coord, t) for coord, t in zip(particle_01_jump_to_a_wrong_well[0, ...], reducedTimeArray)]
max(PE_of_wrong_particle) - min(PE_of_wrong_particle)

In [ ]:
simResult["cfqr"].system.get_potential(init_state_used, 0)

In [ ]:
np.any(all_state[particle_index["10"]][:, :, 1, 0] < 0, axis = 1)

In [ ]:
all_state

In [ ]:
all_state[particle_index["11"], :, 1, 0]

In [ ]:
particle_01_jump_to_a_wrong_well_copy = particle_01_jump_to_a_wrong_well[...]

In [ ]:
particle_01_jump_to_a_wrong_well_copy[:, :, (0, 2, 3), 1] = 0

In [ ]:
particle_01_jump_to_a_wrong_well_copy

In [ ]:
plt.plot(simResult['cfqr'].system.get_kinetic_energy(particle_01_jump_to_a_wrong_well)[0])
plt.plot(simResult['cfqr'].system.get_kinetic_energy(particle_01_jump_to_a_wrong_well_copy)[0])

In [ ]:
particle_01_jump_to_a_wrong_well[0, :, 1, 0]

# old stuff

In [ ]:
particle_index = simResult['cfqr'].getIndexOfParticles()
time_index_start = 0
time_index_end = 500
frame_skip = 5
all_state = simResult['cfqr'].sim.output.all_state['states'][:, time_index_start:time_index_end, ...]
targetTimeArray = protocol_all_time_array[simResult['cfqr'].sim.target_step_index][time_index_start:time_index_end]

animation_time_array = targetTimeArray[::frame_skip]
animation_states = all_state[:, ::frame_skip, ...]
# print("geenerated at ", datetime.datetime.now())

# print(targetTimeArray)
# print(animation_time_array)

fig, ax = plt.subplots(2, 2, figsize=[10, 10])
ax_flatten = ax.flatten()

hist_plot = ax_flatten[0].hist(animation_states[particle_index['00'], 0, 0, 1])[2]
hist_plot = ax_flatten[1].hist(animation_states[particle_index['00'], 0, 1, 1])[2]
# print()

def animate(i):
    params_at_t_str = [f"{key}: {value:.3g}" for key, value in zip(simResult['cfqr'].protocol_key[-6:-1], simResult['cfqr'].protocol.get_params(animation_time_array[i])[-6:-1])]
    fig.suptitle(f"t = {animation_time_array[i]} + {params_at_t_str}")
    ax_flatten[0].clear()
    ax_flatten[1].clear()
    
    ax_flatten[0].hist(animation_states[particle_index['00'], i, 0, 1])
    ax_flatten[1].hist(animation_states[particle_index['00'], i, 1, 1])
            
ani = FuncAnimation(fig, animate, interval=500, frames=len(animation_time_array))
writer = animation.PillowWriter(fps=5, metadata=dict(artist='Me'), bitrate=1800)
ani.save('scatter.gif', writer=writer)
plt.close()


In [ ]:
Image(filename="scatter.gif")

In [ ]:
particle_11_v_y = plt.plot(animation_states[particle_index['11'], :, 1, 1])

In [ ]:
particle_category

In [ ]:
fig, ax1 = plt.subplots(figsize=(7.5, 5))
ax2 = ax1.twinx()

reduced_data = single_particle_analysis(simResult, params, protocol_list, plot_ax = ax1, particle_index = particle_index, particle_category= particle_category[0], time_range=[100, 123])

reduced_protocal_time_array = reduced_data["reduced_protocal_time_array"]
index_slice = reduced_data["index_slice"]

ax2.plot(reduced_protocal_time_array[index_slice], mean_phi_2[index_slice], alpha =0.8, color = "grey", label = f"avg phi_2 of {particle_category}")
ax2.set_ylim(-4, 4)
ax2.set_ylabel(r"$\varphi_{2}$")
y_min, y_max = ax1.set_ylim()
x_min, x_max = ax1.set_xlim()

ax1.set_title(f"energy profile of {particle_category} particle (total time of protocol = {protocol_time_array[-1] * characteristic_time * 1e9:.3g} ns)")

ax2.hlines(1.5, x_min, x_max, color = "red", linestyle = "--")
ax2.hlines(0.5, x_min, x_max, color = "red", linestyle = "--")



# show state at time t

In [ ]:
def search_protocol_time_array_index(array, start):
    x = np.sort(np.append(array, start))
    
    start_index = np.where(x == start)[0][0]
    if start != round(array[start_index]):
        start_index += 1
    return start_index

In [ ]:
import imageio

In [ ]:
all_states_of_00 = all_states[simResult['cfqr'].getIndexOfParticles()["00"]]
all_states_of_01 = all_states[simResult['cfqr'].getIndexOfParticles()["01"]]
all_states_of_10 = all_states[simResult['cfqr'].getIndexOfParticles()["10"]]
all_states_of_11 = all_states[simResult['cfqr'].getIndexOfParticles()["11"]]
np.save("state_for_no_KE_extraction/all_states_of_00.npy", all_states_of_00)
np.save("state_for_no_KE_extraction/all_states_of_01.npy", all_states_of_01)
np.save("state_for_no_KE_extraction/all_states_of_10.npy", all_states_of_10)
np.save("state_for_no_KE_extraction/all_states_of_11.npy", all_states_of_11)
np.save("state_for_no_KE_extraction/reduced_protocol_time_array.npy", reduced_protocal_time_array)


chosen_state = "01"
prefix = "KEP (no)"
if chosen_state == "00":
    plot_states = all_states_of_00
if chosen_state == "01":
    plot_states = all_states_of_01
if chosen_state == "10":
    plot_states = all_states_of_10
if chosen_state == "11":
    plot_states = all_states_of_11
target_index_array = []
animation_picture = []
for t in np.linspace(100, 123, 50):
    target_index = search_protocol_time_array_index(reduced_protocal_time_array, t)
    
    
    if target_index in target_index_array:
        continue
    else:
        target_index_array.append(t)
    
    target_state = plot_states[:, target_index, ...]
    mean_position = np.mean(target_state[:, 0, 0]), np.mean(target_state[:, 1, 0])
    mean_speed = np.mean(target_state[:, 0, 1]), np.mean(target_state[:, 1, 1])
    
    
    fig, ax = plt.subplots(1,1, figsize = [5, 5])
    ax.set_aspect(1)
    ax.scatter(target_state[:, 0, 0], target_state[:, 1, 0])
    ax.scatter(mean_position[0], mean_position[1], color = "red")
    ax.arrow(mean_position[0], mean_position[1], dx = mean_speed[0], dy = mean_speed[1], color = "yellow", width = 0.05)
    ax.set_title(f"particle: {chosen_state}, time = {t:.4g}")
    ax.set_xlim(-4, 4)
    ax.set_ylim(-4, 4)
    
    pic_name = "cfq_data/sudden_jump_KEP/" + f"{prefix}_{chosen_state}_{len(target_index_array)}.png"
    plt.savefig(pic_name)
    plt.close(fig)

    animation_picture.append(imageio.imread(pic_name))

imageio.mimsave(f"cfq_data/sudden_jump_KEP/{prefix}_{chosen_state}_particle.gif", animation_picture, loop = 0, duration=0.1)


In [ ]:
# np.save("saved_init_state/after_CE_completed_N_1000.npy", target_state)

In [ ]:
np.mean(cfqr.system.get_kinetic_energy(target_state))

In [ ]:
reduced_protocal_time_array

# mean position of the particle

In [ ]:
all_states_of_00 = all_states[simResult['cfqr'].getIndexOfParticles()["00"]]

In [ ]:

all_state = simResult['cfqr'].sim.output.all_state['states']
reduced_time_array = simResult['cfqr'].getReducedTimeArray()
index_of_00 = simResult['cfqr'].getIndexOfParticles()["00"]
index_of_01 = simResult['cfqr'].getIndexOfParticles()["01"]
index_of_10 = simResult['cfqr'].getIndexOfParticles()["10"]
index_of_11 = simResult['cfqr'].getIndexOfParticles()["11"]
index_of_not_00 = ~simResult['cfqr'].getIndexOfParticles()["00"]

fig, ax = plt.subplots(4, 4, figsize = [18, 12], sharex=True)

for i, index_of_particle in enumerate([index_of_00, index_of_01, index_of_10, index_of_11]):
    v_1 = np.mean(all_state[index_of_particle, :, 0, 0], axis = 0)
    v_2 = np.mean(all_state[index_of_particle, :, 1, 0], axis = 0)
    v_1dc = np.mean(all_state[index_of_particle, :, 2, 0], axis = 0)
    v_2dc = np.mean(all_state[index_of_particle, :, 3, 0], axis = 0)
    ax[i][0].plot(reduced_time_array, v_1)
    ax[i][1].plot(reduced_time_array, v_2)
    ax[i][2].plot(reduced_time_array, v_1dc)
    ax[i][3].plot(reduced_time_array, v_2dc)
ax[0][0].set_ylabel(r"$\varphi_1$", size = 15)
ax[1][0].set_ylabel(r"$\varphi_2$", size = 15)
ax[2][0].set_ylabel(r"$\varphi_1dc$", size = 15)
ax[3][0].set_ylabel(r"$\varphi_2dc$", size = 15)
fig.supxlabel('time', size = 20)



# KE Analysis

In [ ]:
KE_analysis(simResult)

# v analysis

In [ ]:

all_state = simResult['cfqr'].sim.output.all_state['states']
reduced_time_array = simResult['cfqr'].getReducedTimeArray()
index_of_00 = simResult['cfqr'].getIndexOfParticles()["00"]
index_of_01 = simResult['cfqr'].getIndexOfParticles()["01"]
index_of_10 = simResult['cfqr'].getIndexOfParticles()["10"]
index_of_11 = simResult['cfqr'].getIndexOfParticles()["11"]
index_of_not_00 = ~simResult['cfqr'].getIndexOfParticles()["00"]

fig, ax = plt.subplots(4, 4, figsize = [16, 12])

for i, index_of_particle in enumerate([index_of_00, index_of_01, index_of_10, index_of_11]):
    v_1 = np.mean(all_state[index_of_particle, :, 0, 1], axis = 0)
    v_2 = np.mean(all_state[index_of_particle, :, 1, 1], axis = 0)
    v_1dc = np.mean(all_state[index_of_particle, :, 2, 1], axis = 0)
    v_2dc = np.mean(all_state[index_of_particle, :, 3, 1], axis = 0)
    ax[i][0].plot(reduced_time_array, v_1)
    ax[i][1].plot(reduced_time_array, v_2)
    ax[i][2].plot(reduced_time_array, v_1dc)
    ax[i][3].plot(reduced_time_array, v_2dc)



# KE_of_01 = np.mean(simResult['cfqr'].system.get_kinetic_energy(all_state[index_of_01]), axis = 0)
# KE_of_10 = np.mean(simResult['cfqr'].system.get_kinetic_energy(all_state[index_of_10]), axis = 0)
# KE_of_11 = np.mean(simResult['cfqr'].system.get_kinetic_energy(all_state[index_of_11]), axis = 0)
# # KE_of_10 = np.mean(simResult['cfqr'].system.get_kinetic_energy(all_state[index_of_10]), axis = 0)
# ax[0].plot(protocal_key_time_array, KE_of_00)
# ax[0].set_title("average KE of 00")
# ax[0].set_xlabel("time")
# ax[0].set_ylabel(r"KE $(k_BT)$")
# # ax[0].set_yscale("log")

# ax[1].set_title("average KE of 01")
# ax[1].plot(protocal_key_time_array, KE_of_01)
# ax[1].set_xlabel("time")
# # ax[1].set_ylim(0, 6)

# ax[2].set_title("average KE of 10")
# ax[2].plot(protocal_key_time_array, KE_of_10)
# ax[2].set_xlabel("time")

# ax[3].set_title("average KE of 10")
# ax[3].plot(protocal_key_time_array, KE_of_11)
# ax[3].set_xlabel("time")

# work analysis

In [ ]:
# work_analysis(simResult, params, protocol_list)

### animation

In [ ]:

importlib.reload(coupled_fq_protocol_library)

In [ ]:
def cutlineDataGeneratingFunction(initial_parameter_dict, direction = 'v', color = 'red', guess = [(-2, -2)]):
    coord_index = 0 if direction == "v" else 1
    def tracking_minimum_point(_t):
        return  [cfqr.get_minimum_at_t_for_a_set_of_parameter(_t, initial_parameter_dict, guess = guess)[0][coord_index], direction, color]
    return tracking_minimum_point

In [ ]:
pColor = {"00": "#061DF7", "01": "red", "10": "#3FC7F2", "11": "#F187F4"}

if L_factor == 1:
    graph_setting = {
        0.1: {"vmin": 1000, "vmax": 2000, "ymin": 1000, "ymax": 3000},
        0.5: {"vmin": 100, "vmax": 400, "ymin": 100, "ymax": 400},
        1: {"vmin": 100, "vmax": 400, "ymin": 100, "ymax": 400},
        2: {"vmin": 50, "vmax": 250, "ymin": 50, "ymax": 250},    
        4.2: {"vmin": 600, "vmax": 2000, "ymin": 600, "ymax": 3000},
        5: {"vmin": 0, "vmax": 100, "ymin": 0, "ymax": 150},
        7: {"vmin": 0, "vmax": 100, "ymin": 0, "ymax": 150},
        20: {"vmin": 0, "vmax": 20, "ymin": 0, "ymax": 20},
        100: {"vmin": 0, "vmax": 20, "ymin": 0, "ymax": 20},
        1000: {"vmin": 0, "vmax": 1, "ymin": 0, "ymax": 1},
    }

if L_factor == 1.5:
    graph_setting = {
        0.1: {"vmin": 300, "vmax": 2000, "ymin": 300, "ymax": 2000},
    }

    
if L_factor == 5:
    graph_setting = {
        0.1: {"vmin": -1500, "vmax": 800, "ymin": -1500, "ymax": 800},
    }

if L_1 <=6e-11:
     graph_setting = {
        4.2: {"vmin": 500, "vmax": 3000, "ymin": 500, "ymax": 3000},
    }

In [ ]:
import sys, importlib
importlib.reload(sys.modules['edward_tools.coupled_fq_protocol_library'])
# from edward_tools.coupled_fq_protocol_library import work_analysis, KE_analysis, single_particle_analysis

### without cutlineGeneratingFunction

In [ ]:
contourData = {
    "manual_domain": [np.array([-4, -4]), np.array([4, 4])],
    "contour_range": [700, 2000], "domain_limit": [-3, 3],
    "title": None
}

cutlineInformation = {
    "cutlineList": [(-1.8, "v","red")],
    "cutlineGeneratingFunction": None,
    "cutlineXLimit": [-4,4],
    "cutlineYLimit": [200, 1200]
}

particleInformation = {
    "showParticles": True,
    "project_item": ["00", "01"],
    "project_axis": "phi_2",
    "particle_opacity": 0.5,
    "pColor": {"00": "#061DF7", "01": "red", "10": "#3FC7F2", "11": "#F187F4"}
}



In [ ]:
simResult["cfqr"].getReducedTimeArray()

In [ ]:
time_index_start = 600
time_index_end = 800
frame_skip = 5
all_state = simResult['cfqr'].sim.output.all_state['states'][:, time_index_start:time_index_end, ...]
targetTimeArray = protocol_all_time_array[simResult['cfqr'].sim.target_step_index][time_index_start:time_index_end]

animation_play = True
# animation_play = False
if animation_play:    
    animation_time_array = targetTimeArray[::frame_skip]
    animation_states = all_state[:, ::frame_skip, ...]
    print("geenerated at ", datetime.datetime.now())

    ani, fig, ax = coupled_fq_protocol_library.animate_sim_flux_qubit_with_cutline_and_projection(simResult['cfqr'], time_array=animation_time_array, state_array = animation_states, plot_axis= [0,1], vmin=graph_setting[T]['vmin'], vmax=graph_setting[T]['vmax'], ax0_title = f"T = {T}K, L = {L_1 * 1e12}pH", params=params, 
cutlineGeneratingFunction = False, cutlineInformation = cutlineInformation, particleInformation = particleInformation, contourData = contourData, pColor=pColor, contour_range = [0, 500], cutlineXLimit=[-4, 4], cutlineYLimit=[0, 500])
    fig.tight_layout()
    writer = animation.PillowWriter(fps=10, metadata=dict(artist='Me'), bitrate=1800)
    ani.save('scatter.gif', writer=writer)
    plt.close()
    


In [ ]:
Image(filename="scatter.gif")

In [ ]:
import sys, importlib
importlib.reload(sys.modules['edward_tools.coupled_fq_protocol_library'])
# from edward_tools.coupled_fq_protocol_library import work_analysis, KE_analysis, single_particle_analysis
animation_play = True
# animation_play = False
# contourData = {
#     "manual_domain": [np.array([-4, -4]), np.array([4, 4])],
#     "contour_range": [350, 1500], "domain_limit": [-3, 3],
#     "title": None
# }
contourData = {
    "manual_domain": [np.array([-4, -4]), np.array([4, 4])],
    "contour_range": [700, 1800], "domain_limit": [-3, 3],
    "title": None
}

cutlineInformation = {
    "cutlineList": [(2.3, "v","red")],
    "cutlineGeneratingFunction": None,
    "cutlineXLimit": [-4,4],
    "cutlineYLimit": [200, 1700]
}

particleInformation = {
    "showParticles": True,
    "project_item": ["10", "11"],
    "project_axis": "phi_2",
    "particle_opacity": 0.5,
    "pColor": {"00": "#061DF7", "01": "red", "10": "#3FC7F2", "11": "#F187F4"}
}


In [ ]:

animation_play = True
# animation_play = False
if animation_play:
    frame_skip = 50
    targetTimeArray = protocol_all_time_array[simResult['cfqr'].sim.target_step_index]
    animation_time_array = targetTimeArray[::frame_skip]
    animation_states = all_state[:, ::frame_skip, ...]
    print("geenerated at ", datetime.datetime.now())

    
    ani, fig, ax = coupled_fq_protocol_library.animate_sim_flux_qubit_with_cutline_and_projection(simResult['cfqr'], time_array=animation_time_array, state_array = animation_states, plot_axis= [0,1],
                                                                                   vmin=graph_setting[T]['vmin'], vmax=graph_setting[T]['vmax'], ax0_title = f"T = {T}K, L = {L_1 * 1e12}pH",
                                                                                   params=params, cutlineGeneratingFunction = False, 
                                                                                   cutlineInformation = cutlineInformation, particleInformation = particleInformation, contourData = contourData,
                                                                                   pColor=pColor, contour_range = [0, 500], cutlineXLimit=[-4, 4], cutlineYLimit=[0, 500])
    fig.tight_layout()
    writer = animation.PillowWriter(fps=10, metadata=dict(artist='Me'), bitrate=1800)
    ani.save('scatter.gif', writer=writer)
    plt.close()
    


In [ ]:
Image(filename="scatter.gif")

### with cutlineGeneratingFunction

In [ ]:
animation_play = True
# animation_play = False
if animation_play:

    frame_skip = int(1/50 * 1/params['dt'])
    print("geenerated at ", datetime.datetime.now())
    targetTimeArray = protocol_all_time_array[simResult['cfqr'].sim.target_step_index]
    ani, fig, ax = coupled_fq_protocol_library.animate_sim_flux_qubit_with_cutline_and_projection(simResult['cfqr'], time_array=targetTimeArray, frame_skip = frame_skip, plot_axis= [0,1],
                                                                                   vmin=graph_setting[T]['vmin'], vmax=graph_setting[T]['vmax'], ax0_title = f"T = {T}K, L = {L_1 * 1e12}pH",
                                                                                   params=params, cutlineGeneratingFunction = cutlineDataGeneratingFunction(initial_parameter_dict),
                                                                                   pColor=pColor, contour_range = [0, 500], cutlineXLimit=[-4, 4], cutlineYLimit=[0, 500])
    fig.tight_layout()
    writer = animation.PillowWriter(fps=15, metadata=dict(artist='Me'), bitrate=1800)
    ani.save('scatter.gif', writer=writer)
    Image(filename="scatter.gif")

### save data analysis

In [ ]:
print(f"current comment: {params['comment']}")
response = input("Are you happy with the current comment? If not, give me a new comment.")
if response == "y":
    pass
else:
    params['comment'] = response

In [ ]:
simResult['error_rate_array']

In [ ]:
importlib.reload(sys.modules['edward_tools.cfq_batch_sweep'])

In [ ]:
cfq_batch_sweep.saveSimulationResult(simResult, U0_1, timeOrStep = 'step', save = True, save_final_state = False, saveFolderPath = "coupled_flux_qubit_protocol/coupled_flux_qubit_data_gallery", save_all_state = False)

# save state

In [ ]:
target_state = all_states[:,106, ...]
plt.scatter(target_state[..., 0, 0], target_state[..., 1, 0])

In [ ]:
state_name = "just_before_KEP_N_1000.npy"
np.save("saved_init_state/" + state_name, target_state)

# save pickle

In [ ]:
experiment_label = "Experiment 8 (2025-5-2)" 

In [ ]:
with open(f'coupled_flux_qubit_protocol/simulation_protocol_history/{experiment_label}.pickle', "rb") as f:
    k = pickle.load(f)

In [ ]:
protocol_list

In [ ]:
export_cfqr_params = params['circuit_parameters']

export_params = params
export_params['comment'] = f"{experiment_label}: phi_2xdc = 1.8"


export_object = {
    "cfqr_params": export_cfqr_params,
    "params": export_params,
    "initial_external_parameter_dict": initial_parameter_dict,
    "protocol_list": protocol_list,
    "manual_domain": [np.array([-5, -5]), np.array([5, 5])]
}
    

print(export_params['comment'])
print(simResult['simulation_data']['simulation_id'])

In [ ]:
pickle_file_path = os.path.join("coupled_flux_qubit_protocol", "simulation_protocol_history", f"{experiment_label}.pickle")

with open(pickle_file_path, "wb") as f:
    pickle.dump(export_object, f)

In [ ]:
pickle_file_path

In [ ]:
# modify pickle

In [ ]:
modify_item = [(0, 200), (40, 160), (80, 120), (120, 80), (160, 40), (200, 0)]

In [ ]:
import copy


In [ ]:
k_array = []

In [ ]:
for _i, (t_1, t_2) in enumerate(modify_item):
    k_template = copy.deepcopy(k)

        
    k_template['protocol_list'][1]['duration'] = t_1
    k_template['protocol_list'][2]['duration'] = t_2
    
    if t_1 == 0:
        k_template['protocol_list'].pop(1)
    if t_2 == 0:
        k_template['protocol_list'].pop(2)
    k_array.append(k_template)
    
    pickle_file_path = os.path.join("coupled_flux_qubit_protocol", "simulation_protocol_history", f"{experiment_label}_series_{_i}.pickle")

    with open(pickle_file_path, "wb") as f:
        pickle.dump(k_template, f)
    

In [ ]:
for x in k_array:
    print(x['protocol_list'])

In [ ]:
pickle_prefix = "Experiment 1 (2024-11-12)_series_"
# pickle_prefix = "Experiment 1 (2024-10-27)"
print(pickle_prefix)
comment = f"Experiment 1 (2024-11-12) CE operation, with R = 300Ω and various ratio of the KE protocol"
param_dt = 1/100
loopTime = 100
protocol_type = "CE"






####################
# no need to change the following 
####################

pickle_file_base_folder = "coupled_flux_qubit_protocol/simulation_protocol_history"

all_pickle_file = os.listdir(pickle_file_base_folder)
target_pickle_file_list = [os.path.join(pickle_file_base_folder, p) for p in all_pickle_file if pickle_prefix in p]


In [ ]:
target_pickle_file_list

In [ ]:
particle_index = 0
particle_category = "00"
time_range = None

index_choice = simResult['cfqr'].getIndexOfParticles()["00"]
all_states = simResult['cfqr'].sim.output.all_state['states']
all_states_00_i1 = all_states[index_choice][particle_index, ...]

In [ ]:
_, _, protocol_time_array, protocol_time_index_array = simResult['cfqr'].createProtocolTimeArray(protocol_list, params)

protocol_all_time_index_array = np.array(range(0, int(np.array(protocol_time_array[-1]) / params['dt']) + 1))
protocol_all_time_array  = protocol_all_time_index_array * params['dt']



In [ ]:
reduced_protocal_time_array = simResult['cfqr'].getKeyTime()
KE_of_the_particle = simResult['cfqr'].system.get_kinetic_energy(all_states_00_i1)
PE_of_the_particle = [simResult['cfqr'].system.get_potential(_s, _t ) for _t, _s in  zip(reduced_protocal_time_array, all_states_00_i1)]
WD_of_the_particle = simResult['cfqr'].sim.work_dist_time_array_whole_process[index_choice][particle_index]

In [ ]:
keyState_choosen_particle = [ s[index_choice][particle_index, ...] for s in simResult['cfqr'].sim.keyStep_all_states]
total_energy = PE_of_the_particle - min(PE_of_the_particle) + KE_of_the_particle
adjusted_PE = PE_of_the_particle - PE_of_the_particle[0]
adjusted_total_energy = adjusted_PE + KE_of_the_particle
characteristic_t = np.sqrt(params['circuit_parameters']['L_1'] * params['circuit_parameters']['L_factor'] * params['circuit_parameters']['C_1'] * params['circuit_parameters']['C_factor'])


In [ ]:
reduced_protocal_time_array

In [ ]:
def search_protocol_time_array_index(array, start, end):
    x = np.sort(np.append(array, start))
    y = np.sort(np.append(array, end))
    
    start_index = np.where(x == start)[0][0]
    if start != round(array[start_index]):
        start_index += 1
    
    
    final_index = np.where(y == end)[0][0] 
    if end != round(array[final_index]):
        final_index -= 1

    return slice(start_index, final_index)

In [ ]:
index_slice = search_protocol_time_array_index(reduced_protocal_time_array, 23, 60.)

# reduced_protocal_time_array[index_slice]
# search_protocol_time_array_range(adjusted_PE, 23, 60.)

In [ ]:
reduced_protocal_time_array[index_slice]
KE_of_the_particle[None]

In [ ]:
export_object

In [ ]:
def single_particle_analysis(simResult, params, protocol_list, plot_ax = None, particle_index = 0, particle_category = "00", time_range = None):
    

    
    
    # _ = [simResult['cfqr'].system.get_potential(_s, _t ) for _s, _t in zip(keyState_choosen_particle, simResult['cfqr'].protocol_time_array)]
    
    
    

    if plot_ax == None:
        plt.figure(figsize=(8, 5))
    plot_ax.plot(protocal_key_time_array, adjusted_PE , label = "PE", color = "#CC79A7")
    plot_ax.plot(protocal_key_time_array, KE_of_the_particle, label = "KE")
    plot_ax.plot(protocal_key_time_array, adjusted_total_energy, label = "KE + PE")
    plot_ax.plot(protocal_key_time_array, WD_of_the_particle[::100], label = "WD", color = "#000000")
    plot_ax.plot(protocal_key_time_array, WD_of_the_particle[::100] - adjusted_total_energy, label = "energy loss")
    plot_ax.set_ylabel(r"energy $(k_BT)$")
    plot_ax.set_xlabel(f"time ({characteristic_t * 1e9:.3g} ns)")
    plot_ax.legend(bbox_to_anchor=(1.25, 1.05))

In [ ]:
os.path.join("~", "source", "coupled_flux_qubit_protocol", "simulation_protocol_history", f"{245}.pickle")




In [ ]:
"/Users/tkwtang/source/coupled_flux_qubit_protocol/simulation_protocol_history/"

# circuit parameter at different t and ∆U

In [ ]:
reduced_time_array = cfqr.getReducedTimeArray()

In [ ]:
params_at_t = [cfqr.convert_params_list_into_dict(cfqr.protocol.get_params(_t)) for _t in reduced_time_array]

In [ ]:
circuit_parameters_at_different_t = []

for x in params_at_t:
    circuit_parameter_template = experiment_circuit_params.copy()
    circuit_parameter_template['phi_1x'] = x['phi_1x']
    circuit_parameter_template['phi_2x'] = x['phi_2x']
    circuit_parameter_template['phi_1xdc'] = x['phi_1xdc']
    circuit_parameter_template['phi_2xdc'] = x['phi_2xdc']
    circuit_parameter_template['mu_12'] = x['mu_12']
    circuit_parameters_at_different_t.append(circuit_parameter_template)

In [ ]:
from edward_tools.pot_analysis_helper_functions import get_XYU, pot_function, find_all_critical_points_for_all_potential

In [ ]:
contour_data_array = [get_XYU(circuit_params, domain=[-4, 4], resolution=500) for circuit_params in circuit_parameters_at_different_t]


In [ ]:
initial_guess = [(-2, -2), (-2, 2), (2, -2), (2, 2), (-2,0), (0, -2), (2, 0), (0, 2)]
critical_points_color = ["green", "red", "yellow", "orange", "white", "white", "white", "white"]

In [ ]:
k_BT

In [ ]:
result_array = []
for _cp in circuit_parameters_at_different_t:
    critical_dict = find_all_critical_points_for_all_potential(_cp,
                    guess = [(-2, -2), (-2, 2), (2, -2), (2, 2), \
                             (-2,0), (0, -2), (2, 0), (0, 2)])
    critical_points = list(critical_dict.values())[0]
    critical_potential = [pot_function(_cp)([x, y]) for x, y in critical_points]
    delta_U00 = critical_potential[4] - critical_potential[1]
    delta_U01 = critical_potential[5] - critical_potential[0]
    delta_U10 = critical_potential[6] - critical_potential[2]
    delta_U11 = critical_potential[6] - critical_potential[3]

    X, Y, U = get_XYU(_cp, [-4, 4], 50)

    result_array.append({
        "coord": critical_points, "delta_U00": delta_U00, "delta_U01": delta_U01,
        "delta_U10": delta_U10, "delta_U11": delta_U11
    })


In [ ]:
delta_U_data = [(item['delta_U00'], item['delta_U01'], item['delta_U10'], item['delta_U11'])  for item in result_array]

In [ ]:
total_duration = protocol_time_array[-1] * characteristic_time * 1e9

plt.plot(reduced_time_array * characteristic_time * 1e9, U0_1 * np.array(delta_U_data)[:, 0], label = "∆U00")
plt.plot(reduced_time_array * characteristic_time * 1e9, U0_1 * np.array(delta_U_data)[:, 1], label = "∆U01")
plt.plot(reduced_time_array * characteristic_time * 1e9, U0_1 * np.array(delta_U_data)[:, 2], label = "∆U10")
plt.plot(reduced_time_array * characteristic_time * 1e9, U0_1 * np.array(delta_U_data)[:, 3], label = "∆U11")
xmin, xmax = plt.xlim()
plt.hlines(y = 50, xmin = xmin, xmax = xmax, linestyle = "--")
plt.legend()
plt.xlabel(r'time $(ns)$')
plt.ylabel(r'energy $(k_BT)$')
plt.title(f"∆U00 vs t (total duration = {total_duration:.3g}ns) ")


# save all states

In [ ]:
base_folder = "cfq_data/sudden_jump_KEP/"
prefix = "KEP(jump)"
name_of_state_file = prefix + "_all_state.npy"
name_of_reduced_time_array_file = prefix + "_reduced_time_array.npy"
name_of_index_file = prefix + "_index_of_particle.npy"

np.save(os.path.join(base_folder, name_of_reduced_time_array_file), reduced_protocal_time_array)
np.save(os.path.join(base_folder, name_of_state_file), all_states)
np.save(os.path.join(base_folder, name_of_index_file), simResult['cfqr'].getIndexOfParticles())